# Data analysis

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import urlparse
import urllib
import json
import pandas as pd
import os

driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [/home/marco/.wdm/drivers/chromedriver/linux64/95.0.4638.17/chromedriver] found in cache
/tmp/ipykernel_424193/2406386856.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [2]:
driver.get("https://sarc.asystom.com/monitor/login")

elem = driver.find_element_by_name("user")
elem.clear()
elem.send_keys(os.environ["SITE_USER"])

elem = driver.find_element_by_name("password")
elem.clear()
elem.send_keys(os.environ["SITE_PASSWORD"])

btn = driver.find_element_by_class_name("css-mij01a-button")
btn.click()

/tmp/ipykernel_424193/4134849667.py:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  elem = driver.find_element_by_name("user")
/tmp/ipykernel_424193/4134849667.py:7: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  elem = driver.find_element_by_name("password")
/tmp/ipykernel_424193/4134849667.py:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  btn = driver.find_element_by_class_name("css-mij01a-button")


In [3]:
def get_dfs(parsed_json):
    dfs = []
    # parsed_json["results"][0]["series"][0]["columns"]
    # parsed_json["results"][0]["series"][0]["values"]

    for result in parsed_json["results"]:
        print("Result", result["statement_id"])
        for serie in result["series"]:
            df = pd.DataFrame(serie["values"], columns=serie["columns"])
            df.name = serie["name"]
            dfs.append(df)
    return dfs

In [12]:
from urllib import parse
q="SELECT%20median(%22drift%22)%20FROM%20(SELECT%2020*log10(%22vib_x_vel%22*%22vib_y_vel%22*%22vib_z_vel%22)%20AS%20%22drift%22%20FROM%20%22Signature_Drift%22%20WHERE%20%22device%22%3D%2700-04-a3-0b-00-e9-b5-05%27%20AND%20time%20%3E%3D%20now()%20-%2030d%20AND%20%22mileage%22%252%3D1)%20GROUP%20BY%20time(2h)%20fill(none)%3BSELECT%20moving_average(%22decimated%22%2C%2050)%20FROM%20(SELECT%20median(%22drift%22)%20AS%20%22decimated%22%20FROM%20(SELECT%2020*log10(%22vib_x_vel%22*%22vib_y_vel%22*%22vib_z_vel%22)%20AS%20%22drift%22%20FROM%20%22Signature_Drift%22%20WHERE%20%22device%22%3D%2700-04-a3-0b-00-e9-b5-05%27%20AND%20time%20%3E%3D%20now()%20-%2030d%20AND%20%22mileage%22%252%3D1)%20GROUP%20BY%20time(1m)%20fill(none))"

q="https://sarc.asystom.com/monitor/api/datasources/proxy/8/query?db=asystom_db&q=SELECT%20last(%22drift%22)%20FROM%20(SELECT%2020*log10(%22vib_x_vel%22*%22vib_y_vel%22*%22vib_z_vel%22)%20AS%20%22drift%22%20FROM%20%22Signature_Drift%22%20WHERE%20%22device%22%3D%2700-04-a3-0b-00-e9-b5-05%27%20AND%20time%20%3E%3D%20now()%20-%201d%20AND%20%22mileage%22%252%3D1)%0A&epoch=ms"
q_clean = parse.unquote(q)
q_clean

'https://sarc.asystom.com/monitor/api/datasources/proxy/8/query?db=asystom_db&q=SELECT last("drift") FROM (SELECT 20*log10("vib_x_vel"*"vib_y_vel"*"vib_z_vel") AS "drift" FROM "Signature_Drift" WHERE "device"=\'00-04-a3-0b-00-e9-b5-05\' AND time >= now() - 1d AND "mileage"%2=1)\n&epoch=ms'

In [14]:
# SELECT vib_x_vel, vib_y_vel, vib_z_vel

delay = "200d"

q_clean = f"""
SELECT *
FROM "Signature_Drift"
WHERE "device"=\'00-04-a3-0b-00-e9-b5-05\' AND time >= now() - {delay};

SELECT *
FROM "Signature"
WHERE "device"=\'00-04-a3-0b-00-e9-b5-05\' AND time >= now() - {delay};


SELECT *
FROM "Temperature"
WHERE "device"=\'00-04-a3-0b-00-e9-b5-05\' AND time >= now() - {delay};


SELECT *
FROM "Current_Loop"
WHERE "device"=\'00-04-a3-0b-00-e9-b5-05\' AND time >= now() - {delay};
"""

q_econded = parse.quote(q_clean)
url = f'https://sarc.asystom.com/monitor/api/datasources/proxy/8/query?db=asystom_db&q={q_econded}&epoch=ms'

driver.get(url)
driver.page_source
content = driver.find_element_by_tag_name('pre').text
parsed_json = json.loads(content)
parsed_json
dfs = get_dfs(parsed_json)

/tmp/ipykernel_387047/3831526946.py:30: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  content = driver.find_element_by_tag_name('pre').text


Result 0
Result 1
Result 2
Result 3


In [24]:
with open("all_data.json", "w") as f:
    json.dump(parsed_json, f)

In [19]:
print(dfs[0].name)
dfs[0]

Signature_Drift


,time,GW,client,device,mileage,s_00,s_01,s_02,s_03,s_04,...,vib_y_vel,vib_z_acc,vib_z_f1,vib_z_f2,vib_z_f3,vib_z_kurt,vib_z_peak,vib_z_root,vib_z_vel,vibra_custom
0,1632822027177,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,258,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,1.0,1.0,1.0,1.000000,1.000000,1.0,1.000000,1
1,1632822331552,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,259,0.603113,0.428016,1.520943,0.162509,0.070954,...,1.057239,1.102804,1.0,1.0,1.0,0.930266,0.992188,1.0,1.152778,1
2,1632822626555,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,259,1.068894,0.929274,1.863890,1.574731,0.604257,...,1.106270,1.089286,1.0,1.0,1.0,1.433232,1.603636,1.0,1.103600,1
3,1632822926402,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,259,0.295262,-0.921836,-0.814832,0.993935,0.504692,...,1.054773,1.013333,1.0,1.0,1.0,1.080572,1.076412,1.0,1.035784,1
4,1632823226427,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,259,0.441291,-0.576333,1.248798,2.314946,0.782788,...,1.028589,1.066434,1.0,1.0,1.0,1.477110,1.229960,1.0,1.038660,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8316,1635324162236,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,257,-2.721561,-0.190776,-10.230945,-3.822385,-2.450675,...,0.762026,0.417001,0.0,0.0,0.0,0.650602,0.294454,0.0,0.194458,1
8317,1635324462126,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,256,-3.016823,-1.271115,-9.459602,-4.335088,-2.810025,...,0.657247,0.417001,0.0,0.0,0.0,0.632966,0.228455,0.0,0.155566,1
8318,1635324762026,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,257,-2.524720,-2.156901,-9.271916,-4.513619,-1.560311,...,0.720749,0.417001,0.0,0.0,0.0,0.603787,0.238609,0.0,0.165289,1
8319,1635325062055,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,256,-7.084115,-6.890249,-11.315862,-4.472419,-2.004349,...,0.612796,0.400962,0.0,0.0,0.0,0.615330,0.223379,0.0,0.136121,1


In [16]:
print(dfs[1].name)
dfs[1]

Signature


,time,GW,client,device,mileage,s_00,s_01,s_02,s_03,s_04,...,vib_y_vel,vib_z_acc,vib_z_f1,vib_z_f2,vib_z_f3,vib_z_kurt,vib_z_peak,vib_z_root,vib_z_vel,vibra_custom
0,1630308787973,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,0,-70.608835,-71.288624,-71.528954,-72.222477,-72.794690,...,0.108339,0.006348,0.0,0.0,0.0,3.065537,0.024659,0.0,0.090028,0
1,1630308847953,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,0,-71.000229,-71.844816,-71.519799,-72.007324,-72.817578,...,0.076295,0.006104,0.0,0.0,0.0,3.060960,0.023438,0.0,0.076295,0
2,1630308868190,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,0,-70.306706,-71.922637,-72.487984,-72.142367,-72.199588,...,0.093080,0.006104,0.0,0.0,0.0,3.042649,0.022950,0.0,0.100710,0
3,1630308927926,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,0,-71.593042,-71.657130,-72.716869,-72.259098,-73.073930,...,0.105287,0.006104,0.0,0.0,0.0,2.975509,0.021729,0.0,0.073243,0
4,1630308987909,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,0,-71.158160,-71.458000,-72.158389,-71.586175,-72.877089,...,0.065614,0.006104,0.0,0.0,0.0,2.957198,0.022461,0.0,0.094606,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8509,1635324162236,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,257,-58.651865,-64.188602,-70.599680,-72.094301,-72.883955,...,0.366217,0.006348,0.0,0.0,0.0,3.096056,0.028321,0.0,0.122072,0
8510,1635324462126,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,256,-58.947127,-65.268940,-69.828336,-72.607004,-73.243305,...,0.315862,0.006348,0.0,0.0,0.0,3.012131,0.021973,0.0,0.097658,0
8511,1635324762026,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,257,-58.455024,-66.154726,-69.640650,-72.785534,-71.993591,...,0.346380,0.006348,0.0,0.0,0.0,2.873274,0.022950,0.0,0.103761,0
8512,1635325062055,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,256,-63.014420,-70.888075,-71.684596,-72.744335,-72.437629,...,0.294499,0.006104,0.0,0.0,0.0,2.928206,0.021485,0.0,0.085451,0


In [17]:
print(dfs[2].name)
dfs[2]

Temperature


,time,GW,client,device,value,weekday
0,1630308787973,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,22.305737,Monday
1,1630308847953,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,22.305737,Monday
2,1630308868190,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,22.340423,Monday
3,1630308927926,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,22.340423,Monday
4,1630308987909,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,22.340423,Monday
...,...,...,...,...,...,...
8509,1635324162236,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,19.079934,Wednesday
8510,1635324462126,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,18.975876,Wednesday
8511,1635324762026,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,18.906504,Wednesday
8512,1635325062055,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,18.802446,Wednesday


In [18]:
print(dfs[3].name)
dfs[3]

Current_Loop


,time,GW,client,device,value,weekday
0,1630308787973,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,0.000458,Monday
1,1630308847953,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,0.000458,Monday
2,1630308868190,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,0.000916,Monday
3,1630308927926,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,0.000916,Monday
4,1630308987909,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,0.000458,Monday
...,...,...,...,...,...,...
8509,1635324162236,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,3.949187,Wednesday
8510,1635324462126,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,3.959258,Wednesday
8511,1635324762026,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,3.946899,Wednesday
8512,1635325062055,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,3.945525,Wednesday
